In [0]:
# Set the configuration properties
topic_name = "SentimentalAnalysis"
bootstrap_servers_list = "34.125.251.10:9092,34.125.251.10:9093,34.125.251.10:9094"

In [0]:
starting_offset = "earliest"
consumer_group="Group1"

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType
# Define the schema for the data in the Kafka topic
schema = StructType([
    StructField("key", StringType(), True),
    StructField("value", StringType(), True)
])

In [0]:
df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", bootstrap_servers_list) \
        .option("subscribe", topic_name) \
        .option("startingOffsets", starting_offset) \
        .option("group.id", consumer_group) \
        .load() \
        .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [0]:
from pyspark.sql.functions import from_json, col

schema = "comment STRING, comment_likes INT"
parsed_df = df.select("key",from_json(col("value"), schema).alias("parsed_value"))
result_df = parsed_df.select("key","parsed_value.*")


In [0]:
from pyspark.sql.functions import udf
from textblob import TextBlob


# Define a UDF (user-defined function) to apply sentiment analysis to a comment
def get_sentiment(comment):
    score = TextBlob(comment).sentiment.polarity
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'


In [0]:
# Register the UDF
sentiment_udf = udf(get_sentiment)

In [0]:
# Add a new column 'sentiment_score' to the DataFrame
result_df_new = result_df.withColumn('Sentiment', sentiment_udf(col('comment')))
result_df_new.display()